# Test du modèle de classification des prunes africaines

Ce notebook utilise les fonctions existantes dans le dépôt pour tester le modèle de classification des prunes africaines en utilisant le jeu de données Kaggle "African Plums Dataset".

## 1. Configuration de l'environnement pour Google Colab

Commençons par cloner le dépôt GitHub et configurer l'environnement.

In [ ]:
# Vérifier si nous sommes dans Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules
print(f"Exécution dans Google Colab: {IN_COLAB}")

if IN_COLAB:
    # Cloner le dépôt GitHub
    !git clone https://github.com/CodeStorm-mbe/african-plums-classifier.git
    %cd african-plums-classifier
    
    # Installer les dépendances requises
    !pip install -r requirements.txt
    !pip install kaggle

## 2. Configuration de l'API Kaggle

Pour télécharger le jeu de données Kaggle, nous devons configurer l'API Kaggle si ce n'est pas déjà fait dans les notebooks précédents.

In [ ]:
import os
import sys
import json
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import glob
import random
import seaborn as sns
from sklearn.metrics import confusion_matrix
import zipfile
import shutil

# Ajouter le répertoire courant au chemin pour pouvoir importer nos modules
if IN_COLAB:
    # Dans Colab, nous sommes déjà dans le répertoire du projet
    if "/content/african-plums-classifier" not in sys.path:
        sys.path.append("/content/african-plums-classifier")
else:
    # En local, ajouter le répertoire parent
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

# Importer nos modules personnalisés
from data.data_preprocessing import preprocess_single_image
from models.model_architecture import get_model, TwoStageModel

# Définir les chemins
if IN_COLAB:
    # Dans Colab, créer les répertoires dans le dossier du projet cloné
    DATA_ROOT = "data/raw"
    KAGGLE_DIR = "data/kaggle"
    MODELS_DIR = "models/saved"
    TEST_IMAGES_DIR = "data/test_images"
else:
    # En local
    DATA_ROOT = "../data/raw"
    KAGGLE_DIR = "../data/kaggle"
    MODELS_DIR = "../models/saved"
    TEST_IMAGES_DIR = "../data/test_images"

PLUM_DATA_DIR = os.path.join(DATA_ROOT, "plums")  # Sous-dossier pour les prunes
NON_PLUM_DATA_DIR = os.path.join(DATA_ROOT, "non_plums")  # Sous-dossier pour les non-prunes

# Créer les répertoires s'ils n'existent pas
os.makedirs(PLUM_DATA_DIR, exist_ok=True)
os.makedirs(NON_PLUM_DATA_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(TEST_IMAGES_DIR, exist_ok=True)
os.makedirs(KAGGLE_DIR, exist_ok=True)

# Définir les paramètres
IMG_SIZE = 224
RANDOM_SEED = 42

# Fixer les seeds pour la reproductibilité
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(RANDOM_SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Déterminer le device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation de: {device}")

In [ ]:
# Configuration de l'API Kaggle si nécessaire
if IN_COLAB:
    from google.colab import files
    
    # Vérifier si le fichier kaggle.json existe déjà
    kaggle_config_exists = os.path.exists(os.path.expanduser('~/.kaggle/kaggle.json'))
    
    if not kaggle_config_exists:
        print("Veuillez télécharger votre fichier kaggle.json pour l'authentification Kaggle.")
        print("Vous pouvez le générer sur https://www.kaggle.com/account dans la section 'API'.")
        
        # Télécharger le fichier kaggle.json
        uploaded = files.upload()
        
        # Créer le répertoire .kaggle s'il n'existe pas
        os.makedirs(os.path.expanduser('~/.kaggle'), exist_ok=True)
        
        # Déplacer le fichier kaggle.json vers le répertoire .kaggle
        if 'kaggle.json' in uploaded:
            shutil.move('kaggle.json', os.path.expanduser('~/.kaggle/kaggle.json'))
            # Définir les permissions appropriées
            os.chmod(os.path.expanduser('~/.kaggle/kaggle.json'), 600)
            print("Fichier kaggle.json configuré avec succès.")
        else:
            print("Erreur: Le fichier kaggle.json n'a pas été téléchargé.")
    else:
        print("Le fichier kaggle.json existe déjà.")

## 3. Téléchargement des modèles entraînés (pour Google Colab)

Si vous êtes dans Google Colab et que vous avez déjà entraîné des modèles, vous pouvez les télécharger ici.

In [ ]:
if IN_COLAB:
    # Créer une fonction pour télécharger les modèles
    def upload_models():
        print("Veuillez télécharger les fichiers de modèle suivants:")
        print("1. detection_best_acc.pth")
        print("2. classification_best_acc.pth")
        print("3. two_stage_model_info.json")
        
        # Télécharger les fichiers
        uploaded = files.upload()
        
        # Déplacer les fichiers vers le répertoire des modèles
        for filename in uploaded.keys():
            shutil.move(filename, os.path.join(MODELS_DIR, filename))
            print(f"Fichier {filename} déplacé vers {MODELS_DIR}")
    
    # Vérifier si les modèles existent déjà
    model_files_exist = (
        os.path.exists(os.path.join(MODELS_DIR, 'detection_best_acc.pth')) and
        os.path.exists(os.path.join(MODELS_DIR, 'classification_best_acc.pth')) and
        os.path.exists(os.path.join(MODELS_DIR, 'two_stage_model_info.json'))
    )
    
    if not model_files_exist:
        print("Les fichiers de modèle n'existent pas. Vous pouvez les télécharger maintenant.")
        # Décommentez la ligne suivante pour télécharger les modèles
        # upload_models()
    else:
        print("Les fichiers de modèle existent déjà.")

## 4. Préparation des images de test à partir du jeu de données Kaggle

Téléchargeons et préparons des images de test à partir du jeu de données Kaggle "African Plums Dataset".

In [ ]:
def download_kaggle_dataset(force_download=False):
    """Télécharge le jeu de données Kaggle 'African Plums Dataset'."""
    # Vérifier si le jeu de données a déjà été téléchargé
    dataset_zip = os.path.join(KAGGLE_DIR, 'african-plums-dataset.zip')
    if os.path.exists(dataset_zip) and not force_download:
        print(f"Le jeu de données a déjà été téléchargé dans {dataset_zip}.")
        return dataset_zip
    
    print("Téléchargement du jeu de données Kaggle 'African Plums Dataset'...")
    try:
        # Télécharger le jeu de données
        !kaggle datasets download -d arnaudfadja/african-plums-quality-and-defect-assessment-data -p {KAGGLE_DIR}
        print(f"Jeu de données téléchargé avec succès dans {dataset_zip}.")
        return dataset_zip
    except Exception as e:
        print(f"Erreur lors du téléchargement du jeu de données: {e}")
        return None

def extract_and_prepare_test_images(dataset_zip, force_extract=False, num_images_per_class=3):
    """Extrait et prépare des images de test à partir du jeu de données Kaggle."""
    if not os.path.exists(dataset_zip):
        print(f"Le fichier {dataset_zip} n'existe pas.")
        return False
    
    # Vérifier si les données ont déjà été extraites
    extracted_dir = os.path.join(KAGGLE_DIR, 'extracted')
    if os.path.exists(extracted_dir) and not force_extract:
        print(f"Le jeu de données a déjà été extrait dans {extracted_dir}.")
    else:
        print(f"Extraction du jeu de données...")
        os.makedirs(extracted_dir, exist_ok=True)
        
        # Extraire le fichier zip
        with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
            zip_ref.extractall(extracted_dir)
        
        print(f"Jeu de données extrait avec succès dans {extracted_dir}.")
    
    # Préparer des images de test
    print("Préparation des images de test...")
    
    # Vérifier la structure du jeu de données extrait
    dataset_dir = os.path.join(extracted_dir, 'african_plums_dataset')
    if not os.path.exists(dataset_dir):
        print(f"Le répertoire {dataset_dir} n'existe pas.")
        return False
    
    # Mapper les classes du jeu de données Kaggle
    classes = ['bruised', 'cracked', 'rotten', 'spotted', 'unaffected', 'unripe']
    
    # Sélectionner des images aléatoires pour chaque classe
    for cls in classes:
        src_dir = os.path.join(dataset_dir, cls)
        if not os.path.exists(src_dir):
            print(f"Le répertoire {src_dir} n'existe pas.")
            continue
        
        # Obtenir la liste des images
        images = [f for f in os.listdir(src_dir) if os.path.isfile(os.path.join(src_dir, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        if not images:
            print(f"Aucune image trouvée dans {src_dir}.")
            continue
        
        # Sélectionner des images aléatoires
        selected_images = random.sample(images, min(num_images_per_class, len(images)))
        
        # Copier les images sélectionnées vers le répertoire de test
        for i, img_name in enumerate(selected_images):
            src_path = os.path.join(src_dir, img_name)
            dst_path = os.path.join(TEST_IMAGES_DIR, f"test_{cls}_{i+1}.jpg")
            shutil.copy(src_path, dst_path)
            print(f"Image copiée: {src_path} -> {dst_path}")
    
    # Ajouter des images non-prune (si disponibles)
    non_plum_dir = os.path.join(NON_PLUM_DATA_DIR, "non_plum")
    if os.path.exists(non_plum_dir):
        non_plum_images = [f for f in os.listdir(non_plum_dir) if os.path.isfile(os.path.join(non_plum_dir, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        if non_plum_images:
            selected_non_plum = random.sample(non_plum_images, min(5, len(non_plum_images)))
            for i, img_name in enumerate(selected_non_plum):
                src_path = os.path.join(non_plum_dir, img_name)
                dst_path = os.path.join(TEST_IMAGES_DIR, f"test_non_plum_{i+1}.jpg")
                shutil.copy(src_path, dst_path)
                print(f"Image non-prune copiée: {src_path} -> {dst_path}")
    
    # Vérifier le nombre d'images de test préparées
    test_images = [f for f in os.listdir(TEST_IMAGES_DIR) if os.path.isfile(os.path.join(TEST_IMAGES_DIR, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    print(f"\n{len(test_images)} images de test préparées avec succès.")
    
    return True

def prepare_test_images(force_prepare=False, num_images_per_class=3):
    """Prépare des images de test à partir du jeu de données Kaggle ou crée des images synthétiques."""
    # Vérifier si des images de test existent déjà
    existing_images = [f for f in os.listdir(TEST_IMAGES_DIR) if os.path.isfile(os.path.join(TEST_IMAGES_DIR, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    if existing_images and not force_prepare:
        print(f"Des images de test existent déjà ({len(existing_images)} images). Utilisez force_prepare=True pour les remplacer.")
        return
    
    # Essayer de télécharger et préparer des images de test à partir du jeu de données Kaggle
    try:
        dataset_zip = download_kaggle_dataset(force_download=False)
        if dataset_zip:
            success = extract_and_prepare_test_images(dataset_zip, force_extract=False, num_images_per_class=num_images_per_class)
            if success:
                print("Images de test préparées avec succès à partir du jeu de données Kaggle.")
                return
    except Exception as e:
        print(f"Erreur lors de la préparation des images de test à partir du jeu de données Kaggle: {e}")
    
    # Si le téléchargement ou la préparation échoue, créer des images synthétiques
    print("Création d'images de test synthétiques...")
    
    # Définir les couleurs pour les différentes classes
    colors = {
        "ripe": (150, 0, 0),      # Rouge foncé
        "unripe": (0, 150, 0),    # Vert foncé
        "damaged": (150, 100, 0), # Marron
        "diseased": (100, 0, 100),# Violet
        "overripe": (100, 0, 0),  # Rouge très foncé
        "healthy": (150, 50, 50)  # Rouge-rose
    }
    
    # Créer des images pour chaque classe de prune
    for cls, base_color in colors.items():
        for i in range(3):  # 3 images par classe
            # Ajouter un peu de variation aléatoire à la couleur
            color_var = [max(0, min(255, c + random.randint(-20, 20))) for c in base_color]
            
            # Créer une image
            img = Image.new('RGB', (224, 224), (255, 255, 255))
            pixels = img.load()
            
            # Dessiner un cercle approximatif avec la couleur
            center_x, center_y = 112, 112
            radius = 100 + random.randint(-10, 10)
            
            for x in range(img.width):
                for y in range(img.height):
                    dist = ((x - center_x) ** 2 + (y - center_y) ** 2) ** 0.5
                    if dist <= radius:
                        # Ajouter du bruit à chaque pixel
                        pixel_color = [max(0, min(255, c + random.randint(-10, 10))) for c in color_var]
                        pixels[x, y] = tuple(pixel_color)
            
            # Sauvegarder l'image
            img_path = os.path.join(TEST_IMAGES_DIR, f"test_{cls}_{i+1}.jpg")
            img.save(img_path)
    
    # Créer des images non-prune
    for i in range(5):  # 5 images non-prune
        # Couleur aléatoire qui n'est pas proche des couleurs de prune
        color = (random.randint(0, 100), random.randint(150, 255), random.randint(150, 255))
        
        # Créer une image
        img = Image.new('RGB', (224, 224), (255, 255, 255))
        pixels = img.load()
        
        # Dessiner une forme aléatoire (carré ou triangle)
        shape = random.choice(['square', 'triangle'])
        
        if shape == 'square':
            # Dessiner un carré
            size = random.randint(100, 150)
            top_left = (random.randint(0, 224-size), random.randint(0, 224-size))
            
            for x in range(top_left[0], top_left[0] + size):
                for y in range(top_left[1], top_left[1] + size):
                    if 0 <= x < 224 and 0 <= y < 224:
                        # Ajouter du bruit à chaque pixel
                        pixel_color = [max(0, min(255, c + random.randint(-10, 10))) for c in color]
                        pixels[x, y] = tuple(pixel_color)
        else:
            # Dessiner un triangle
            p1 = (random.randint(50, 174), random.randint(50, 174))
            p2 = (p1[0] + random.randint(30, 50), p1[1] + random.randint(30, 50))
            p3 = (p1[0] - random.randint(0, 30), p2[1])
            
            # Remplir le triangle (algorithme simple)
            min_x = min(p1[0], p2[0], p3[0])
            max_x = max(p1[0], p2[0], p3[0])
            min_y = min(p1[1], p2[1], p3[1])
            max_y = max(p1[1], p2[1], p3[1])
            
            for x in range(min_x, max_x + 1):
                for y in range(min_y, max_y + 1):
                    if 0 <= x < 224 and 0 <= y < 224:
                        # Vérification simple si le point est dans le triangle
                        if (x >= p1[0] and y >= p1[1] and x <= p2[0] and y <= p2[1]):
                            # Ajouter du bruit à chaque pixel
                            pixel_color = [max(0, min(255, c + random.randint(-10, 10))) for c in color]
                            pixels[x, y] = tuple(pixel_color)
        
        # Sauvegarder l'image
        img_path = os.path.join(TEST_IMAGES_DIR, f"test_non_plum_{i+1}.jpg")
        img.save(img_path)
    
    # Vérifier le nombre d'images de test créées
    test_images = [f for f in os.listdir(TEST_IMAGES_DIR) if os.path.isfile(os.path.join(TEST_IMAGES_DIR, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    print(f"\n{len(test_images)} images de test synthétiques créées avec succès.")

# Préparer des images de test
prepare_test_images(force_prepare=False, num_images_per_class=3)

## 5. Visualisation des images de test

Visualisons les images de test que nous avons préparées.

In [ ]:
def visualize_test_images():
    """Visualise les images de test."""
    # Obtenir la liste des images de test
    test_images = [f for f in os.listdir(TEST_IMAGES_DIR) if os.path.isfile(os.path.join(TEST_IMAGES_DIR, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    if not test_images:
        print("Aucune image de test trouvée.")
        return
    
    # Trier les images par classe
    test_images.sort()
    
    # Déterminer le nombre de lignes et de colonnes pour l'affichage
    num_images = min(len(test_images), 15)  # Limiter à 15 images pour l'affichage
    num_cols = 5
    num_rows = (num_images + num_cols - 1) // num_cols
    
    # Créer la figure
    plt.figure(figsize=(15, 3 * num_rows))
    
    # Afficher chaque image
    for i, img_name in enumerate(test_images[:num_images]):
        img_path = os.path.join(TEST_IMAGES_DIR, img_name)
        img = Image.open(img_path)
        
        plt.subplot(num_rows, num_cols, i + 1)
        plt.imshow(img)
        plt.title(img_name.replace('test_', '').replace('.jpg', ''))
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

# Visualiser les images de test
visualize_test_images()

## 6. Chargement du modèle

Chargeons le modèle à deux étapes que nous avons entraîné précédemment.

In [ ]:
def load_two_stage_model():
    """Charge le modèle à deux étapes à partir des fichiers sauvegardés."""
    # Vérifier si les fichiers nécessaires existent
    model_info_path = os.path.join(MODELS_DIR, 'two_stage_model_info.json')
    detection_model_path = os.path.join(MODELS_DIR, 'detection_best_acc.pth')
    classification_model_path = os.path.join(MODELS_DIR, 'classification_best_acc.pth')
    
    if not os.path.exists(model_info_path):
        raise FileNotFoundError(f"Le fichier d'informations du modèle n'existe pas: {model_info_path}")
    if not os.path.exists(detection_model_path):
        raise FileNotFoundError(f"Le fichier du modèle de détection n'existe pas: {detection_model_path}")
    if not os.path.exists(classification_model_path):
        raise FileNotFoundError(f"Le fichier du modèle de classification n'existe pas: {classification_model_path}")
    
    # Charger les informations du modèle
    with open(model_info_path, 'r') as f:
        model_info = json.load(f)
    
    # Extraire les informations
    detection_classes = model_info['detection_classes']
    classification_classes = model_info['classification_classes']
    model_architecture_info = model_info['model_info']
    
    # Créer les modèles
    detection_model = get_model(
        model_name=model_architecture_info['detection_model']['base_model'].split('_')[0] if '_' in model_architecture_info['detection_model']['base_model'] else 'standard',
        num_classes=len(detection_classes),
        base_model=model_architecture_info['detection_model']['base_model'].split('_')[1] if '_' in model_architecture_info['detection_model']['base_model'] else model_architecture_info['detection_model']['base_model'],
        pretrained=False
    )
    
    classification_model = get_model(
        model_name=model_architecture_info['classification_model']['base_model'].split('_')[0] if '_' in model_architecture_info['classification_model']['base_model'] else 'standard',
        num_classes=len(classification_classes),
        base_model=model_architecture_info['classification_model']['base_model'].split('_')[1] if '_' in model_architecture_info['classification_model']['base_model'] else model_architecture_info['classification_model']['base_model'],
        pretrained=False
    )
    
    # Charger les poids
    detection_model.load_state_dict(torch.load(detection_model_path, map_location=device))
    classification_model.load_state_dict(torch.load(classification_model_path, map_location=device))
    
    # Créer le modèle à deux étapes
    two_stage_model = TwoStageModel(
        detection_model, 
        classification_model, 
        detection_threshold=model_architecture_info['detection_threshold'] if 'detection_threshold' in model_architecture_info else 0.7
    )
    
    return two_stage_model, detection_classes, classification_classes

# Charger le modèle
try:
    model, detection_classes, classification_classes = load_two_stage_model()
    model.detection_model.to(device)
    model.classification_model.to(device)
    
    print("Modèle à deux étapes chargé avec succès!")
    print(f"Classes de détection: {detection_classes}")
    print(f"Classes de classification: {classification_classes}")
except Exception as e:
    print(f"Erreur lors du chargement du modèle: {e}")
    print("\nSi vous n'avez pas encore entraîné le modèle, veuillez d'abord exécuter le notebook d'entraînement.")
    
    # Créer des classes fictives pour pouvoir continuer le notebook
    detection_classes = ['plum', 'non_plum']
    classification_classes = ['bruised', 'cracked', 'rotten', 'spotted', 'unaffected', 'unripe']
    print("\nUtilisation de classes fictives pour la démonstration:")
    print(f"Classes de détection: {detection_classes}")
    print(f"Classes de classification: {classification_classes}")

## 7. Prédiction sur une image individuelle

Testons notre modèle sur une image individuelle.

In [ ]:
def predict_single_image(model, img_path, detection_classes, classification_classes):
    """Prédit la classe d'une image individuelle."""
    # Vérifier si le fichier existe
    if not os.path.exists(img_path):
        print(f"Le fichier {img_path} n'existe pas.")
        return None, None, None
    
    # Charger et prétraiter l'image
    img = Image.open(img_path).convert('RGB')
    img_tensor = preprocess_single_image(img, IMG_SIZE)
    img_tensor = img_tensor.unsqueeze(0).to(device)  # Ajouter la dimension du batch
    
    # Passer en mode évaluation
    model.detection_model.eval()
    model.classification_model.eval()
    
    # Prédire
    with torch.no_grad():
        # Prédiction de détection
        detection_output = model.detection_model(img_tensor)
        detection_probs = torch.nn.functional.softmax(detection_output, dim=1)
        detection_pred = torch.argmax(detection_probs, dim=1).item()
        detection_prob = detection_probs[0, detection_pred].item()
        
        # Si c'est une prune, prédire la classe
        if detection_classes[detection_pred] == 'plum' and detection_prob >= model.detection_threshold:
            # Prédiction de classification
            classification_output = model.classification_model(img_tensor)
            classification_probs = torch.nn.functional.softmax(classification_output, dim=1)
            classification_pred = torch.argmax(classification_probs, dim=1).item()
            classification_prob = classification_probs[0, classification_pred].item()
            
            return detection_classes[detection_pred], classification_classes[classification_pred], {
                'detection_prob': detection_prob,
                'classification_prob': classification_prob,
                'detection_probs': detection_probs.cpu().numpy()[0],
                'classification_probs': classification_probs.cpu().numpy()[0]
            }
        else:
            return detection_classes[detection_pred], None, {
                'detection_prob': detection_prob,
                'detection_probs': detection_probs.cpu().numpy()[0]
            }

def visualize_prediction(img_path, detection_pred, classification_pred, probs):
    """Visualise l'image avec sa prédiction."""
    # Charger l'image
    img = Image.open(img_path).convert('RGB')
    
    # Créer la figure
    plt.figure(figsize=(10, 6))
    
    # Afficher l'image
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.title("Image de test")
    plt.axis('off')
    
    # Afficher les prédictions
    plt.subplot(1, 2, 2)
    plt.axis('off')
    
    # Texte de prédiction
    text = f"Détection: {detection_pred} ({probs['detection_prob']:.2f})\n"
    if classification_pred is not None:
        text += f"Classification: {classification_pred} ({probs['classification_prob']:.2f})"
    else:
        text += "Classification: N/A"
    
    plt.text(0.1, 0.5, text, fontsize=12)
    
    # Afficher les probabilités de détection
    if 'detection_probs' in probs:
        plt.figure(figsize=(10, 4))
        plt.subplot(1, 2, 1)
        plt.bar(detection_classes, probs['detection_probs'])
        plt.title("Probabilités de détection")
        plt.xticks(rotation=45)
        
        # Afficher les probabilités de classification si disponibles
        if 'classification_probs' in probs:
            plt.subplot(1, 2, 2)
            plt.bar(classification_classes, probs['classification_probs'])
            plt.title("Probabilités de classification")
            plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()

# Tester le modèle sur une image individuelle
if 'model' in locals():
    # Obtenir la liste des images de test
    test_images = [f for f in os.listdir(TEST_IMAGES_DIR) if os.path.isfile(os.path.join(TEST_IMAGES_DIR, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    if test_images:
        # Sélectionner une image aléatoire
        test_img = random.choice(test_images)
        test_img_path = os.path.join(TEST_IMAGES_DIR, test_img)
        
        print(f"Test du modèle sur l'image: {test_img}")
        detection_pred, classification_pred, probs = predict_single_image(model, test_img_path, detection_classes, classification_classes)
        
        if detection_pred is not None:
            print(f"Prédiction de détection: {detection_pred} (probabilité: {probs['detection_prob']:.2f})")
            if classification_pred is not None:
                print(f"Prédiction de classification: {classification_pred} (probabilité: {probs['classification_prob']:.2f})")
            else:
                print("Prédiction de classification: N/A")
            
            # Visualiser la prédiction
            visualize_prediction(test_img_path, detection_pred, classification_pred, probs)
        else:
            print("Erreur lors de la prédiction.")
    else:
        print("Aucune image de test trouvée.")

## 8. Prédiction sur toutes les images de test

Testons notre modèle sur toutes les images de test.

In [ ]:
def predict_all_test_images(model, test_dir, detection_classes, classification_classes):
    """Prédit la classe de toutes les images de test."""
    # Obtenir la liste des images de test
    test_images = [f for f in os.listdir(test_dir) if os.path.isfile(os.path.join(test_dir, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    if not test_images:
        print("Aucune image de test trouvée.")
        return None
    
    # Prédire pour chaque image
    results = []
    for img_name in test_images:
        img_path = os.path.join(test_dir, img_name)
        
        # Extraire la vraie classe à partir du nom de fichier
        true_class = None
        for cls in classification_classes:
            if cls in img_name:
                true_class = cls
                break
        
        if 'non_plum' in img_name:
            true_detection = 'non_plum'
            true_class = None
        else:
            true_detection = 'plum'
        
        # Prédire
        detection_pred, classification_pred, probs = predict_single_image(model, img_path, detection_classes, classification_classes)
        
        # Ajouter aux résultats
        results.append({
            'img_name': img_name,
            'img_path': img_path,
            'true_detection': true_detection,
            'true_class': true_class,
            'detection_pred': detection_pred,
            'classification_pred': classification_pred,
            'probs': probs
        })
    
    return results

def visualize_test_results(results):
    """Visualise les résultats des prédictions sur les images de test."""
    if not results:
        print("Aucun résultat à visualiser.")
        return
    
    # Calculer les métriques de détection
    detection_correct = sum(1 for r in results if r['true_detection'] == r['detection_pred'])
    detection_accuracy = detection_correct / len(results)
    
    # Calculer les métriques de classification
    classification_results = [r for r in results if r['true_detection'] == 'plum' and r['detection_pred'] == 'plum']
    classification_correct = sum(1 for r in classification_results if r['true_class'] == r['classification_pred'])
    classification_accuracy = classification_correct / len(classification_results) if classification_results else 0
    
    # Afficher les métriques
    print(f"Résultats sur {len(results)} images de test:")
    print(f"Précision de détection: {detection_accuracy:.2f} ({detection_correct}/{len(results)})")
    print(f"Précision de classification: {classification_accuracy:.2f} ({classification_correct}/{len(classification_results)})")
    
    # Créer une matrice de confusion pour la détection
    detection_true = [r['true_detection'] for r in results]
    detection_pred = [r['detection_pred'] for r in results]
    detection_classes_unique = sorted(list(set(detection_true + detection_pred)))
    
    detection_cm = confusion_matrix(detection_true, detection_pred, labels=detection_classes_unique)
    
    plt.figure(figsize=(10, 8))
    plt.subplot(2, 1, 1)
    sns.heatmap(detection_cm, annot=True, fmt='d', cmap='Blues', xticklabels=detection_classes_unique, yticklabels=detection_classes_unique)
    plt.title('Matrice de confusion - Détection')
    plt.ylabel('Vraie classe')
    plt.xlabel('Classe prédite')
    
    # Créer une matrice de confusion pour la classification
    if classification_results:
        classification_true = [r['true_class'] for r in classification_results]
        classification_pred = [r['classification_pred'] for r in classification_results]
        classification_classes_unique = sorted(list(set([c for c in classification_true + classification_pred if c is not None])))
        
        classification_cm = confusion_matrix(classification_true, classification_pred, labels=classification_classes_unique)
        
        plt.subplot(2, 1, 2)
        sns.heatmap(classification_cm, annot=True, fmt='d', cmap='Blues', xticklabels=classification_classes_unique, yticklabels=classification_classes_unique)
        plt.title('Matrice de confusion - Classification')
        plt.ylabel('Vraie classe')
        plt.xlabel('Classe prédite')
    
    plt.tight_layout()
    plt.show()
    
    # Afficher quelques exemples de prédictions
    num_examples = min(5, len(results))
    plt.figure(figsize=(15, 4 * num_examples))
    
    for i in range(num_examples):
        result = results[i]
        img = Image.open(result['img_path']).convert('RGB')
        
        plt.subplot(num_examples, 2, 2*i + 1)
        plt.imshow(img)
        plt.title(f"Image: {result['img_name']}")
        plt.axis('off')
        
        plt.subplot(num_examples, 2, 2*i + 2)
        plt.axis('off')
        
        # Texte de prédiction
        text = f"Vraie détection: {result['true_detection']}\n"
        text += f"Prédiction de détection: {result['detection_pred']} ({result['probs']['detection_prob']:.2f})\n\n"
        
        if result['true_class'] is not None:
            text += f"Vraie classe: {result['true_class']}\n"
        else:
            text += "Vraie classe: N/A\n"
        
        if result['classification_pred'] is not None:
            text += f"Prédiction de classification: {result['classification_pred']} ({result['probs']['classification_prob']:.2f})"
        else:
            text += "Prédiction de classification: N/A"
        
        plt.text(0.1, 0.5, text, fontsize=12)
    
    plt.tight_layout()
    plt.show()

# Tester le modèle sur toutes les images de test
if 'model' in locals():
    print("Test du modèle sur toutes les images de test...")
    results = predict_all_test_images(model, TEST_IMAGES_DIR, detection_classes, classification_classes)
    
    if results:
        visualize_test_results(results)

## 9. Test interactif avec téléchargement d'images (pour Google Colab)

Si vous êtes dans Google Colab, vous pouvez télécharger vos propres images pour les tester.

In [ ]:
if IN_COLAB and 'model' in locals():
    from google.colab import files
    
    def test_uploaded_image():
        print("Veuillez télécharger une image à tester.")
        uploaded = files.upload()
        
        for filename in uploaded.keys():
            # Sauvegarder l'image téléchargée
            img_path = os.path.join(TEST_IMAGES_DIR, filename)
            with open(img_path, 'wb') as f:
                f.write(uploaded[filename])
            
            print(f"Test du modèle sur l'image téléchargée: {filename}")
            detection_pred, classification_pred, probs = predict_single_image(model, img_path, detection_classes, classification_classes)
            
            if detection_pred is not None:
                print(f"Prédiction de détection: {detection_pred} (probabilité: {probs['detection_prob']:.2f})")
                if classification_pred is not None:
                    print(f"Prédiction de classification: {classification_pred} (probabilité: {probs['classification_prob']:.2f})")
                else:
                    print("Prédiction de classification: N/A")
                
                # Visualiser la prédiction
                visualize_prediction(img_path, detection_pred, classification_pred, probs)
            else:
                print("Erreur lors de la prédiction.")
    
    # Décommentez la ligne suivante pour tester avec vos propres images
    # test_uploaded_image()

## 10. Conclusion

Dans ce notebook, nous avons utilisé les fonctions existantes des modules `data_preprocessing` et `model_architecture` pour :
1. Télécharger et préparer des images de test à partir du jeu de données Kaggle "African Plums Dataset"
2. Charger le modèle à deux étapes entraîné précédemment
3. Tester le modèle sur des images individuelles
4. Évaluer les performances du modèle sur un ensemble d'images de test
5. Tester le modèle sur des images téléchargées par l'utilisateur (dans Google Colab)

Le modèle est maintenant prêt à être utilisé pour classifier des prunes africaines en fonction de leur qualité et de leurs défauts.